In [1]:
import librosa
import numpy as np
import pandas as pd 
import glob
import joblib

In [2]:
WAVE_OUTPUT_FILENAME = "test_file.wav"
de_label = pd.read_csv('de_train_label.csv', engine='python', index_col = None)

In [3]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [4]:
def parse_audio_files(filenames):
    rows = len(filenames)
    # feature는 각 파일 별 row(window) * 피처 의 2차원 행렬
    # labels은 파일 별 카테고리 int 값
    features = np.zeros((rows,193))
    i = 0
    for fn in filenames:
        try:
            mfccs, chroma, mel, contrast,tonnetz = extract_feature(fn)
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        except:
            print("error : "+fn)
        else:
            features[i] = ext_features
            print("success...")
            i += 1
    return features

In [5]:
files = []
files.extend(glob.glob(WAVE_OUTPUT_FILENAME))
X_test = parse_audio_files(files)

success...


In [8]:
X_test = X_test.reshape(X_test.shape[0], 193, 1)

In [9]:
model = joblib.load('model_2.pkl')
pred = model.predict_proba(X_test)
print(pred)

Using TensorFlow backend.


[[20.17857]]


C:\Users\jih02\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\sequential.py:248: UserWarning: Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).
  warnings.warn('Network returning invalid probability values. '


In [10]:
ans = round(float(pred))
de_label = pd.read_csv('de_train_label.csv', engine='python', index_col = None)
print("이번 역은 "+de_label['name'][ans]+"역 입니다.")

이번 역은 신도림역 입니다.
